# 2D labels to 3D labels
This notebook loads a label image from a 2D OME-Zarr image, as well as a ROI table.
It then converts it to a 3D segmentation and saves that to the OME-Zarr file. Also saving a new ROI_table there

In [ ]:
import zarr
import dask.array as da
import numpy as np
from skimage.measure import label
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square, remove_small_holes, remove_small_objects
import napari
from ome_zarr.writer import write_labels
from napari_ome_zarr_roi_loader.utils import read_table
from pathlib import Path
from anndata.experimental import write_elem

In [ ]:
zarr_url = "/Users/joel/Desktop/20230510_Fractal_web_test/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr/B/03/0"
zarr_3D_url = "/Users/joel/Desktop/20230510_Fractal_web_test/20200812-CardiomyocyteDifferentiation14-Cycle1.zarr/B/03/0"
new_z_planes = 19
level = 0
label_name = "nuclei"
ROI_tables_to_copy = ["FOV_ROI_table"]

In [ ]:
# TODO: Check the case where Z spacing is not 1um: 
# What is in the scale parameter then? Is it also set to e.g. 2, 0.16, 0.16? in 2D .zattrs?
# Load the Z pixel size from there
z_pixel_size = 1.0

### 1) Load a 2D label image

In [ ]:
label_img = da.from_zarr(f"{zarr_url}/labels/{label_name}/{level}")
with zarr.open(f"{zarr_url}/labels/{label_name}", mode='rw+') as zarr_label_img:
    coordinate_transforms_label_img = zarr_label_img.attrs["multiscales"][0]['datasets']
    label_img_scale = zarr_label_img.attrs['multiscales'][0]['datasets'][level]['coordinateTransformations'][0]["scale"]

### 2) Create a 3D stack of the label image

In [ ]:
label_img_3D = da.stack([label_img.squeeze()] * new_z_planes)
label_img_3D

In [ ]:
# Have a look at the label image in napari
# Needs the numpy arrays, because dask arrays aren't easily edited in napari
viewer = napari.Viewer()
viewer.add_image(np.array(img), scale=img_scale)
# label_layer = viewer.add_labels(label_image)
# Optionally set a correct scale (or load it from the OME-Zarr file):
label_layer = viewer.add_labels(np.array(label_img), scale=label_img_scale)

In [ ]:
# Optionally modify the label layer manually in napari, then get that modified label layer
label_image = label_layer.data

### 3b) Save changed label image to OME-Zarr

In [ ]:
# TODO: Make this just the old label name (once debugging is done)
new_label_name = "manual_label_correction_5"
chunks = (1, 2160, 2560)
with zarr.open(zarr_3D_url, mode='rw+') as zarr_img:
    write_labels(
        label_img_3D,
        zarr_img,
        name = new_label_name,
        axes = "zyx",
        chunks = chunks,
        storage_options = {"dimension_separator": '/'}
    )

    # Hacky way of ensuring we have the correct metadata, because the writer 
    # doesn't get the metadata right yet.
    # This assumes the output labels have the same shape as the loaded image
    # coordinate_transforms = zarr_img.attrs["multiscales"][0]['datasets']
    axes = zarr_img.attrs["multiscales"][0]["axes"]
    labels_zarr = zarr_img[f"labels/{new_label_name}"]
    multiscales = labels_zarr.attrs['multiscales']
    multiscales[0]['datasets'] = coordinate_transforms_label_img
    # Skip the channels axis, because this image contains no channels
    multiscales[0]["axes"] = axes[1:]
    labels_zarr.attrs['multiscales'] = multiscales

    # Update ROI tables
    for ROI_table in ROI_tables_to_copy:
        roi_an = read_table(Path(zarr_url), ROI_table)
        nb_rois = len(roi_an.X)
        # Set the new Z values to span the whole ROI
        roi_an.X[:, 5] = np.array([z_pixel_size*new_z_planes]*nb_rois)

        # TODO: Check that the table doesn't exist yet. 
        # Otherwise make an overwrite check?
    
        # Save the ROI table to the 3D OME-Zarr file
        new_roi_name = ROI_table
        group_tables = zarr_img.require_group("tables/")
        write_elem(group_tables, new_roi_name, roi_an)
        
        # Update the tables .zattrs for the new table
        group_tables.attrs["tables"] = group_tables.attrs["tables"] + [new_roi_name]

In [ ]:
import anndata as ad
table_url = Path(zarr_3D_url) / f"tables/{new_roi_name}"
roi_an = ad.read_zarr(table_url)

In [ ]:
roi_df = roi_an.to_df()
roi_df

In [ ]:
# DONE: Also modify & copy over the ROI table: 
# Only if there are new ROI tables, e.g. an organoid ROI table?
# => input parameter which ROI_tables to copy over

In [ ]:
# TODO: Test on Silvia's data

In [ ]:
# TODO: Wrap this up in a task
# change the metadata:
# Changing metadata needs a second task atm, because it needs to be plate parallel, not per well